In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add the parent directory to the Python path
parent_dir = os.path.abspath('../..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import modules
import ddsp_textures.loss.functions
import ddsp_textures.auxiliar.filterbanks

# Import extra packages
import numpy as np
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
import torch
import time

In [2]:
print("Fire statistics:\n")
names = ["stats_11", "stats_12", "stats_13", "stats_14", "stats_2", "stats_3", "stats_4", "stats_5"]

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load audio to filter -----------------------------------
fire_path  = ".."+"/sounds/fire_sounds/fire.wav"
water_path = ".."+"/sounds/water_sounds/water.wav"
sr     = 44100
new_sr = sr // 4 # for log_bank
fire_audio, _  = librosa.load(fire_path, sr=sr)
fire_audio = fire_audio/np.max(np.abs(fire_audio))
water_audio, _ = librosa.load(water_path, sr=sr)
water_audio = water_audio/np.max(np.abs(water_audio))
# Make list of segments for fire and water --------------
frame_size     = 2**15
new_frame_size = frame_size // 4
hop_size   = 2**15
fire_segments = []
water_segments = []
for i in range(0, len(fire_audio)-frame_size, hop_size):
    # segment_norm = (fire_audio[i:i+frame_size] - np.mean(fire_audio[i:i+frame_size])) / np.std(fire_audio[i:i+frame_size])
    segment = fire_audio[i:i+frame_size]
    fire_segments.append(segment)
for i in range(0, len(water_audio)-frame_size, hop_size):
    # segment_norm = (water_audio[i:i+frame_size] - np.mean(water_audio[i:i+frame_size])) / np.std(water_audio[i:i+frame_size])
    segment = water_audio[i:i+frame_size]
    water_segments.append(segment)
# Initialize erb_bank and log_bank for statistics loss --
N_filter_bank = 16
M_filter_bank = 6
erb_bank    = ddsp_textures.auxiliar.filterbanks.EqualRectangularBandwidth(frame_size, sr, N_filter_bank, 20, sr // 2)
log_bank    = ddsp_textures.auxiliar.filterbanks.Logarithmic(new_frame_size,       new_sr, M_filter_bank, 10, new_sr // 4)

import torchaudio
downsampler = torchaudio.transforms.Resample(sr, new_sr).to(device)

Fire statistics:



In [ ]:
#Testing Statistics functions --------------------------------------------------
fire_segment = fire_segments[0]

# create tensor with req gra
input_tensor = torch.tensor(fire_segment, requires_grad=True).to(device)
output_1 = ddsp_textures.loss.functions.statistics_mcds(      input_tensor, N_filter_bank, M_filter_bank, erb_bank, log_bank, downsampler)
output_2 = ddsp_textures.loss.functions.statistics_mcds_old(  input_tensor, N_filter_bank, M_filter_bank, erb_bank, log_bank, downsampler)

# Compare gradients for each tensor in the list
for i in range(5):
    print("Difference in stats", i, (torch.sort(output_1[i].flatten()).values - torch.sort(output_2[i].flatten()).values).abs().max())


Difference in stats 0 tensor(9.0122e-05, device='cuda:0', grad_fn=<MaxBackward1>)
Difference in stats 1 tensor(2.3305e-05, device='cuda:0', grad_fn=<MaxBackward1>)
Difference in stats 2 tensor(8.9407e-08, device='cuda:0', grad_fn=<MaxBackward1>)
Difference in stats 3 tensor(0.0001, device='cuda:0', grad_fn=<MaxBackward1>)
Difference in stats 4 tensor(6.8784e-05, device='cuda:0', grad_fn=<MaxBackward1>)


In [ ]:
#Testing Statistics functions --------------------------------------------------
fire_segment   = fire_segments[0]
fire_segment = np.random.rand(len(fire_segment))
x_1 = torch.tensor(fire_segment, requires_grad=True).to(device)
x_2 = torch.tensor(fire_segment, requires_grad=True).to(device)

# x_1
stats_1, stats_2, stats_3, stats_4, stats_5 = ddsp_textures.loss.functions.statistics_mcds(x_1, N_filter_bank, M_filter_bank, erb_bank, log_bank, downsampler)

stats_1_mean = torch.mean(stats_1)
print("stats_1_mean: ", stats_1_mean)
stats_2_mean = torch.mean(stats_2)
print("stats_2_mean: ", stats_2_mean)
stats_3_mean = torch.mean(stats_3)
print("stats_3_mean: ", stats_3_mean)
stats_4_mean = torch.mean(stats_4)
print("stats_4_mean: ", stats_4_mean)
stats_5_mean = torch.mean(stats_5)
print("stats_5_mean: ", stats_5_mean)

y = [stats_1_mean, stats_2_mean, stats_3_mean, stats_4_mean, stats_5_mean]

#compute time
start = time.time()
# Compute gradients for each output element separately
for j in range(len(y)):
    # For each y[j], compute gradient and retain graph for subsequent iterations
    grad = torch.autograd.grad(y[j], x_1, retain_graph=True)[0]
    print("grad ",names[j],": ", grad)
end = time.time()
print("Time: ", end-start)

stats_1, stats_2, stats_3, stats_4, stats_5 = ddsp_textures.loss.functions.statistics_mcds_old(x_2, N_filter_bank, M_filter_bank, erb_bank, log_bank, downsampler)

stats_1_mean = torch.mean(stats_1)
stats_2_mean = torch.mean(stats_2)
stats_3_mean = torch.mean(stats_3)
stats_4_mean = torch.mean(stats_4)
stats_5_mean = torch.mean(stats_5)

y = [stats_1_mean, stats_2_mean, stats_3_mean, stats_4_mean, stats_5_mean]

#compute time
start = time.time()
# Compute gradients for each output element separately
for j in range(len(y)):
    # For each y[j], compute gradient and retain graph for subsequent iterations
    grad = torch.autograd.grad(y[j], x_2, retain_graph=True)[0]
    print("grad ",names[j],": ", grad)
end = time.time()
print("Time: ", end-start)

stats_1_mean:  tensor(18.4363, device='cuda:0', grad_fn=<MeanBackward0>)
stats_2_mean:  tensor(-0.0088, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
stats_3_mean:  tensor(0.3259, device='cuda:0', grad_fn=<MeanBackward0>)
stats_4_mean:  tensor(-0.0103, device='cuda:0', grad_fn=<MeanBackward0>)
stats_5_mean:  tensor(0.1067, device='cuda:0', grad_fn=<MeanBackward0>)
grad  stats_11 :  tensor([-1.2133e-03,  1.1966e-03,  1.7662e-03,  ..., -3.4601e-04,
        -1.7098e-03, -9.6810e-05], device='cuda:0', dtype=torch.float64)
grad  stats_12 :  tensor([1.8123e-05, 4.2602e-05, 6.1529e-05,  ..., 4.9376e-05, 6.4953e-05,
        3.7321e-05], device='cuda:0', dtype=torch.float64)
grad  stats_13 :  tensor([8.7516e-05, 4.3566e-05, 4.0512e-05,  ..., 6.2467e-05, 9.4924e-05,
        4.2309e-05], device='cuda:0', dtype=torch.float64)
grad  stats_14 :  tensor([ 3.2572e-04, -1.0719e-04, -9.1848e-05,  ..., -1.0835e-04,
         2.6639e-04, -7.0325e-05], device='cuda:0', dtype=torch.float64)


In [5]:
batch_size = 5
batch_example_1 = torch.zeros(batch_size, frame_size).to(device)
for i in range(batch_size):
    batch_example_1[i] = torch.tensor(100*fire_segments[np.random.randint(len(fire_segments))]).to(device)

batch_example_2 = torch.zeros(batch_size, frame_size).to(device)
for i in range(batch_size):
    batch_example_2[i] = torch.tensor(100*water_segments[np.random.randint(len(water_segments))]).to(device)

start=time.time()
loss_new = ddsp_textures.loss.functions.statistics_mcds_loss(    batch_example_1, batch_example_2, N_filter_bank, M_filter_bank, erb_bank, log_bank, downsampler).to(device)
print("Time new: ", time.time()-start)
start=time.time()
loss_old = ddsp_textures.loss.functions.statistics_mcds_loss_old(batch_example_1, batch_example_2, N_filter_bank, M_filter_bank, erb_bank, log_bank, downsampler).to(device)
print("Time old: ", time.time()-start)

print("Loss new: ", loss_new)
print("Loss old: ", loss_old)

Time new:  0.08807706832885742
Time old:  2.1002843379974365
Loss new:  tensor(1095.9083, device='cuda:0')
Loss old:  tensor(1095.9072, device='cuda:0')


In [6]:
# import torch

# def f(x):
#     y = torch.stack([x, x**2, x**3])
#     return y

# x = torch.tensor(5., requires_grad=True)
# y = f(x)

# # Compute gradients for each output element separately
# gradients = []
# for j in range(len(y)):
#     # For each y[j], compute gradient and retain graph for subsequent iterations
#     grad = torch.autograd.grad(y[j], x, retain_graph=True)[0]
#     gradients.append(grad)

# # Convert list of gradients to a tensor
# dy_dx = torch.tensor(gradients)

# print(dy_dx)  # Output: tensor([ 1., 10., 75.])

In [7]:
# import torch

# def f(x):
#     return x  # Identity function: f(x) = [x₁, x₂, x₃]

# # Vector input with 3 elements
# x = torch.tensor([1., 2., 3.], requires_grad=True)
# y = f(x)

# # Compute gradients for each output y_j = x_j
# gradients = []
# for j in range(len(y)):
#     # Compute gradient of y[j] w.r.t x (vector input)
#     grad = torch.autograd.grad(y[j], x, retain_graph=True)[0]
#     gradients.append(grad)

# # Stack gradients to form the Jacobian matrix
# jacobian = torch.stack(gradients)

# print(jacobian)